# Introduction
<p>The collection and use of data for football analysis is an emerging and rapidly developing sector. Describing aspects of an highly random sport like football through data is certainly an ambitious goal, but more and more teams and companies are investing in this field, recognising its enormous potential. Unfortunately, the availability of free open data providing modern performance metrics is still very limited. So, after a careful research, we have selected the following resources in order to build a dataset containing the most important performance metrics and more: Understat, Api-football and Fbref.

We decide not to establish at first a structure for our dataset, but we will explore the available data and then assess which structure is the most appropriate.

Below is the code written to carry out the data acquisition operations and some data cleaning operations in the download phase. We dedicate a section to each data resource.</p>

# Understat

<p>Understat is a website that provides statistics of teams and players from Premier League (England and Wales), Serie A (Italy), La Liga (Spain), Bundesliga (Germany), Ligue 1 (France) and Russian Premier League (Russia), starting from the 2014/2015 season to the actual season. We consider the seasons from 2014/2015 to 2020/2021 of the top 5 football leagues (Premier League, Serie A, La Liga, Bundesliga and Ligue 1).
We chose Understat as a data source because it provides modern and specific performance statistics in football. In order to obtain the data, a special Python package is made available. In particular, to download the data we use the function get_league_players. This function receives an integer indicating the season and a string indicating the league name. It returns a list of dictionaries, one dictionary for each player containing team, position, games played and various game statistics for each player are provided.</p>

<p>We discard players who have played less than 15 games in each season, because we consider a season with less than 15 games played non-evaluable. For the moment we decide to download all the data provided, we only remove the field containing the player's Understat id. We proceed with some simple data cleaning and normalisation operations directly in the download phase. Some numeric data are supplied with values in a string format. We identify the fields with this problem and convert the value types accordingly. Players who changed teams throughout the season, while remaining in the same league, present a double value for the team field and aggregate statistics. We decide to keep only the statistics of the first of the two teams. We consider this to be an acceptable approximation, as for the application that this dataset was conceived the team is of little relevance, while the league is far more important. In order to have an indication of the impact of this approximation, we obtain the percentage of players that present this characteristic. They are 0.03% of the total number of players.<p/>

<p>We generate a json file for each season of a given league, resulting in 35 json files (7 seasons, 5 leagues). We select the options encoding='utf-8' and ensure_ascii=False in order to handle special characters correctly. We insert a specific provision for the apostrophe character while downloading.</p>

<p>Since we want to download data of players from several sources, we need to find a field that will allow us to aggregate the different datasets. We think that the most suitable field is the name of the players, then we save in a separate structure the names of all considered players. We want to get a list of lists, a sub list will contain only the players for each given season and league. In addition to the names we also save the teams they belong to, because it might be useful to solve ambiguities. </p>

<p>Then after we obtained the list containing the names of the players, we add that list to another list called football_players. So once we get a list of lists, where each sublist contains the names of players from a certain league in a certain season, we proceed in the same way with the players' teams, obtaining also in this case a list of lists called players_teams. The usefulness and practical application of this procedure will become clearer later on.</p>


In [ ]:
!pip install understat

     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 144 kB 54.6 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 47.4 MB/s 
     |████████████████████████████████| 280 kB 76.5 MB/s 
     |████████████████████████████████| 280 kB 68.1 MB/s 
     |████████████████████████████████| 280 kB 45.6 MB/s 
     |████████████████████████████████| 280 kB 69.9 MB/s 
     |████████████████████████████████| 279 kB 62.8 MB/s 
     |████████████████████████████████| 279 kB 56.6 MB/s 
     |████████████████████████████████| 272 kB 57.2 MB/s 
     |████████████████████████████████| 272 kB 59.3 MB/s 
     |████████████████████████████████| 272 kB 45.0 MB/s 
INFO: pip is looking at multiple versions of pytest-aiohttp to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 213 kB 73.4 MB/s 
     |████████████████████████████████|

In [ ]:
import asyncio
import aiohttp
import json
import understat
import pandas as pd
from understat import Understat
import nest_asyncio
nest_asyncio.apply()
from difflib import SequenceMatcher

In [ ]:
#the leagues and seasons considered
leagues = ['epl', 'Serie A', 'La Liga', 'Bundesliga', 'Ligue 1']
seasons = [2014, 2015, 2016, 2017, 2018, 2019, 2020]

#the numeric fields
numeric_key_list = ['id', 'games', 'time', 'goals', 'xG', 'assists', 'xA', 'shots', 'key_passes',
 'yellow_cards', 'red_cards', 'npg', 'npxG', 'xGChain', 'xGBuildup']

#list where we are going to add the player names
football_players = []
#the list where we are going to add the lists with player teams
players_teams = []

#To verify how much players contains two values for the field team_title
players_same_team = []
players_total = []

for season in seasons:
  for league in leagues:

    #the list where we are going to add the dictionaires of the players
    res=[]
    #the list where we are going to add the players of a given season and league
    players_seas_league = []
    #the list where we are going to add the teams of a given season and league
    players_teams_league = []

    async def main():
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:
            understat = Understat(session)
            players = await understat.get_league_players(
                league_name=league,
                season=season
            )

            for i in range(len(players)):
              #numeric statistics should be numeric
              for key in numeric_key_list:
                players[i][key] = float(players[i][key])
                players[i][key] = round(players[i][key], 2)

              if(players[i]['games'] >= 15):
                players_total.append('1')
                #players with double value for team field
                if players[i]['team_title'].find(',') != -1:
                  teams = players[i]['team_title'].split(',')
                  players[i]['team_title'] = teams[0]
                  players_same_team.append('1')

                players[i].pop('id')

                if players[i]['player_name'].find('\'') != -1:
                  players[i]['player_name'] = players[i]['player_name'].replace('&#039;', '\'')

                #we add name of players to the players_seas_league list
                players_seas_league.append(players[i]['player_name'])
                #we add the team of players to the players_teams_leagues list
                players_teams_league.append(players[i]['team_title'])

                #we add the dictionaire containing the data of players to the res list
                element = dict(players[i])
                res.append(element)
              else:
                continue

            #After analyzing each players of a given league for a given season, we add the res list to a json file
            # with open('stats_' + league + str(season) + '_players.json', 'a', encoding='utf-8') as outfile:
            #   json.dump(res, outfile, ensure_ascii=False)

            #we add the built sub list to the football_players and players_teams list
            football_players.append(players_seas_league)
            players_teams.append(players_teams_league)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

#we print the percentage of players with double value in the team_title field
print(len(players_same_team)/len(players_total))


0.030521969920377468


# Api-football
<p>Api-football is a site that provides an api, called api-football-beta, which allows you to obtain data on players, teams, coaches, transfers and betting odds for various leagues around the world. The number of seasons available depends on the league. The game statistics are much simpler than the ones provided by Understat, so we decide to use this api only to obtain players' age, nationality, weight and height. Here again, we consider the seasons from 2014/2015 to 2020/2021 of the Premier League, Serie A, La Liga, Bundesliga and Ligue 1.</p>

<p>We provide 2 strings as input parameters to the api, the first one indicating the season and the other one indicating the league id. The data available for a certain league and season are provided on several pages, so in the download stage we add a third string indicating the page. With an initial request we get the total number of pages available for a certain league and season and visit them all. The return value is a dictionary of dictionaries and lists. The fields we are interested in are contained in the player dictionary within the response list. We obtain a dictionary for each player.</p>

<p>Again, we only consider players with more than 15 games in the season and implement some data cleaning operations directly in the download phase. Weight and height have string values, so we convert them into numerical values. The height is given in cm, the weight in kg. The age field indicates the current age of the player, however we are interested in the age of the player in a given season. We obtain this value from the birth date field. Moreover, we delete the fields of the player dictionary which are not of interest to us. </p>

<p>As before, we generate a json file for each season of a given league, resulting in 35 json files (7 seasons, 5 leagues). We select the options encoding='utf-8' and ensure_ascii=False in order to treat special characters correctly.
</p>

<p>This data will need to be aggregated with the data downloaded from Understat. In order to do this, we need to make sure that each players have a field with the same value in both datasets, so that we can use that field as a key for aggregation. The field that seems most suitable for this operation is the name of the players. For each player, when downloading the data from api-football, we assign to the name field a name taken from the football_players list containing the names of the players downloaded from Understat. In this way the names are the same in both datasets. We explain below how we correctly associate the names of the players between the two datasets.</p>

<p>We write a checkname function which, given the name of a player downloaded from api-football, determines the name of the corresponding player from the data downloaded from Understat. The implementation of this function is complex since the same player may be represented with syntactically different names in the two sites. The function we have implemented receives:

 * the players_teams sublist of the appropriate season and year (downloaded from Understat)
 * the football_players sublist of the appropriate season and year (downloaded from Understat),
 * the full name of the player (downloaded from api-football)
 * the common name (downloaded from api-football, name field)
 * the team (downloaded from api-football, team field)

This function analyzes each string in football_players and calculates its similarity (using the SequenceMatcher criterion) with the full name and with the common name. The SequenceMatcher criterion is an implementation of the LCS distance that does not consider spaces and punctuation. We choose this distance, as opposed to e.g. edit distance, because we believe that differences in names are syntactic and not due to types. A value of 0.2 is added to the similarity value between the players names if the similarity between the team names overcome a predefined threshold (if the name and the team are similar, it is very likely that the player has been correctly recognised and we do not want to risk losing the association). If the maximum similarity is above a predefined threshold (0.8) then the corresponding string is returned, otherwise an empty string is returned and the corresponding player will not be included in the constructed dataset. The values of the thresholds were chosen after some testing. The commented code of the function is given below.</p>

<p>We decided to implement this process directly in the download phase in order to facilitate the subsequent data aggregation process.</p>

<p>We bring a key for the api as an example. The free subscription plan only allows 100 requests per day, so running the code at some point will lead to an error because more than 100 requests are needed to download all the data.</p>

In [ ]:
def checkname(players, name, nickname, squadre_understat, squadra_api):
  maximum = 0
  maximum_2 = 0

  res = ''
  res_2 = ''

  for i in range(len(players)):
    #if the selected players of the Understat list contains name and surname the comparison will be done with the full name
    if(len(players[i].split(' ')) > 1):
      if(maximum < SequenceMatcher(None, players[i], name).ratio()):
        plus = 0
        #if in addition to the name the similarity is also in the name of the team a plus will be added
        if(SequenceMatcher(None, squadre_understat[i], squadra_api).ratio() > 0.7):
          plus = plus + 0.2
        maximum = SequenceMatcher(None, players[i], name).ratio() + plus
        res = players[i]
    #.. otherwise the comparison will be done with the common name, important trick for players like 'Isco' or 'Koke'
    else:
      nick =  SequenceMatcher(None, players[i], nickname).ratio()
      if(maximum_2 < nick ):
        plus = 0
        if (SequenceMatcher(None, squadre_understat[i], squadra_api).ratio() > 0.7):
          plus = plus + 0.2
        maximum_2 =  nick + plus
        res_2 = players[i]

  if(maximum > 0.8): #priority to a possible similarity with the full name
    return res
  else:
    if(maximum_2 > 0.8):
      return res_2
    else:
      return ''

In [ ]:
import requests
import json

leagues_ids = ["39", "135", "140", "78", "61"]
seasons = ["2014", "2015", "2016", "2017", "2018", "2019", "2020"]
#iteration index that we are going to use to give the football_players and players_teams sublist
iteration = 0

key = ''

for season in seasons:
  for league_id, league in zip(leagues_ids, leagues):

    #list where we are going to add the downloaded dictionaires
    res = []

    #we obtain the number of pages for a given season and league
    url = "https://api-football-beta.p.rapidapi.com/players"
    querystring = {"season": season , "league": league_id}
    headers = {
        'x-rapidapi-host': "api-football-beta.p.rapidapi.com",
        'x-rapidapi-key':  key
        }
    result = requests.request("GET", url, headers=headers, params=querystring)
    diz=result.json()
    pages = diz['paging']['total']

    for page in range(1, pages + 1):
      page = str(page)

      url = "https://api-football-beta.p.rapidapi.com/players"
      querystring = {"season":season,"league":league_id, "page":page}
      headers = {
          'x-rapidapi-host': "api-football-beta.p.rapidapi.com",
          'x-rapidapi-key':  key
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      aDict = response.json()
      jsonStr = json.dumps(aDict, ensure_ascii=False)
      diz = json.loads(jsonStr, encoding='utf-8')

      for j in range(len(diz['response'])):
          if diz['response'][j]['statistics'][0]['games']['appearences'] is not None:
            if(diz['response'][j]['statistics'][0]['games']['appearences'] >= 15):

              #we obtain the first name and last name
              first_name = diz['response'][j]['player']['firstname'].split(' ')
              second_name = diz['response'][j]['player']['lastname'].split(' ')
              name = first_name[0]
              #trick to deal surname like De Bruyne or Di Maria
              if(len(second_name[0]) > 2):
                surname = second_name[0]
              else:
                if(len(second_name) > 1):
                  surname = second_name[0] + second_name[1]
                else:
                  surname = second_name[0]

              player_name = name + ' ' + surname
              #we assaign the return value of checkname to the name field
              diz['response'][j]['player']['name'] = checkname(football_players[iteration], player_name,
                                                               diz['response'][j]['player']['name'],
                                                               players_teams[iteration], diz['response'][j]['statistics'][0]['team']['name'])

              if(diz['response'][j]['player']['name'] != ''):
                #we obtain the age of the players in each given season
                year_birth = diz['response'][j]['player']['birth']['date'][:4]
                diz['response'][j]['player']['age'] = int(season) - int(year_birth)

                #we remove the fields that we are not interested in
                diz['response'][j].pop('statistics')
                diz['response'][j]['player'].pop('birth')
                diz['response'][j]['player'].pop('firstname')
                diz['response'][j]['player'].pop('lastname')
                diz['response'][j]['player'].pop('injured')
                diz['response'][j]['player'].pop('id')

                #we transform height and weight in numeric values
                if isinstance(diz['response'][j]['player']['height'], str):
                  diz['response'][j]['player']['height'] = diz['response'][j]['player']['height'][:-2]
                  diz['response'][j]['player']['height'] = int(diz['response'][j]['player']['height'])
                if isinstance(diz['response'][j]['player']['weight'], str):
                  diz['response'][j]['player']['weight'] = diz['response'][j]['player']['weight'][:-2]
                  diz['response'][j]['player']['weight'] = int(diz['response'][j]['player']['weight'])

                #we add the dictionaires of the players to the res list
                element = dict(diz['response'][j]['player'])
                res.append(element)
              else:
                continue
    iteration = iteration + 1
    #analyzed each players of a league for a given season, we add the res list in a json file
    with open(season + league + '_players.json', 'a', encoding='utf-8') as outfile:
        json.dump(res, outfile, ensure_ascii=False)

# Fbref

<p> Fbref is a site that provides a large amount of very modern performance statistics. Compared to Understat it covers many more aspects of the game by providing specific data (e.g. passes or defensive actions). However, it does have some issues: it does not provide a free API, the links do not have a fixed structure suitable for web scraping and the data is only available from the 2017/2018 season onwards. Despite this, we feel it is essential to obtain performance statistics in addition to those from Understat to improve our dataset. Therefore we decide to reduce the number of seasons that our dataset will be able to cover in order to increase the completeness of the data. The data are in a tabular format and available for download in csv format, but we decide to use web scraping to be able to select the data more easily and obtain json files immediately. To implement the web scraping we have two options: get a url provided by Fbref that sends directly to the table of interest, simplifying the code, or use the url of the page where the table of interest is located. We choose the second option because although it complicates the code, it is faster to copy the links to the pages of interest than to obtain the links to the tables. </p>

<p>We take data on players passing and defensive actions. For both aspects of the game, we select the metrics that seem most meaningful to us from those available. Again, it is necessary to convert the values of some fields from strings to numerical values. We normalise the values in the Pos column and keep only the first value. We decide that a player can only have one role, because we think this is the best solution for the purpose of our dataset. We consider only the first role indicated.</p>

<p>Using the same process as above with a few minor changes we ensure that the player names are the same as those provided by Understat. We rewrite the football_players list so that it starts from the 2017 season in order to use the same index matching mechanism. We modify the checkname function, which in this case receives one string less, and does not receive the full name of the player because Fbref does not provide it. It is not necessary to use the players_teams list in this case because the name formats are already quite similar.</p>

<p>We generate a json file for each season and league, so we end up with 20 json files (5 leagues, 4 seasons).</p>

In [ ]:
#we make football_players starting from 2017/2018 season
football_players = football_players[15:]
players_teams = players_teams[15:]

def checkname(players, name, squadre_understat, squadra_fbref):
  maximum = 0

  res = ''

  if(len(name.split(' ')) > 1):
    first_name = name.split(' ')[0]
    last_name = name.split(' ')[1]

    for i in range(len(players)):
      massimo = max(SequenceMatcher(None, players[i], name).ratio(), SequenceMatcher(None, players[i], first_name).ratio(), SequenceMatcher(None, players[i], last_name).ratio())
      if(maximum < massimo):
        plus = 0
        if(SequenceMatcher(None, squadre_understat[i], squadra_fbref).ratio() > 0.7):
          plus = plus + 0.2

        maximum = massimo
        res = players[i]
  else:
    for i in range(len(players)):
      if(maximum < SequenceMatcher(None, players[i], name).ratio()):
        plus = 0
        if(SequenceMatcher(None, squadre_understat[i], squadra_fbref).ratio() > 0.7):
          plus = plus + 0.2

        maximum = SequenceMatcher(None, players[i], name).ratio() + plus
        res = players[i]

  if(maximum > 0.8):
    return res
  else:
    return ''

In [ ]:
!pip install selenium
!pip install bs4

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

In [ ]:
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
#url of interest
defensive_links =['https://fbref.com/en/comps/9/1631/defense/2017-2018-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/1640/defense/2017-2018-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/1652/defense/2017-2018-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/1634/defense/2017-2018-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/1632/defense/2017-2018-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/1889/defense/2018-2019-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/1896/defense/2018-2019-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/1886/defense/2018-2019-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/2109/defense/2018-2019-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/2104/defense/2018-2019-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/3232/defense/2019-2020-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/3260/defense/2019-2020-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/3239/defense/2019-2020-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/3248/defense/2019-2020-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/3243/defense/2019-2020-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/10728/defense/2020-2021-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/10730/defense/2020-2021-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/10731/defense/2020-2021-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/10737/defense/2020-2021-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/10732/defense/2020-2021-Ligue-1-Stats']

iteration = 0
for link in defensive_links:
    page = driver.get(link)
    time.sleep(2)

    #we find the table
    table = driver.find_element(By.XPATH , '//*[@id="stats_defense"]')
    table_html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(table_html, "html.parser")

    #here are the headers columns of the dataframe where we will copy the table
    headers = []
    for i in soup.find_all('th'):
        title = i.text
        headers.append(title)
    headers = headers[9:40]
    df=pd.DataFrame(columns=headers)

    #we build the dataset
    for j in soup.find_all('tr')[2:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        if row:
            length = len(df)
            df.loc[length] = row

    #we assaign the return value of checkname to the name field, we keep only one role
    for i in range(len(df)-1):
        name = df.loc[i, 'Player']
        team = df.loc[i, 'Squad']
        df.loc[i, 'Player'] = checkname(football_players[iteration], name, players_teams[iteration], team)
        pos = df.loc[i, 'Pos']
        if(len(pos.split(',')) > 1):
            df.loc[i, 'Pos'] = pos.split(',')[0]

    df = df[['Player', 'Pos', 'Tkl', 'TklW', 'Past', 'Press', 'Succ', 'Blocks', 'Int']]
    df.columns = ['Player', 'Pos', 'Tkl', 'Tkl2', 'TklW', 'Past', 'Press', 'Succ', 'Blocks', 'Int']

    for i in range(len(df) - 1):
        if df.loc[i, 'Tkl']:
            df.loc[i, 'Tkl'] = float(df.loc[i, 'Tkl'])
        if df.loc[i, 'TklW']:
            df.loc[i, 'TklW'] = float(df.loc[i, 'TklW'])
        if df.loc[i, 'Past']:
            df.loc[i, 'Past'] = float(df.loc[i, 'Past'])
        if df.loc[i, 'Press']:
            df.loc[i, 'Press'] = float(df.loc[i, 'Press'])
        if df.loc[i, 'Succ'] :
            df.loc[i, 'Succ'] = float(df.loc[i, 'Succ'])
        if df.loc[i, 'Blocks']:
            df.loc[i, 'Blocks'] = float(df.loc[i, 'Blocks'])
        if df.loc[i, 'Int']:
            df.loc[i, 'Int'] = float(df.loc[i, 'Int'])


    iteration = iteration + 1

    df.drop(df[df.Player == ''].index, inplace=True)
    #we keep only the statistics that we are interested in
    df = df[['Player', 'Pos', 'Tkl', 'TklW', 'Past', 'Press', 'Succ', 'Blocks', 'Int']]

    link = link.split('/')[-1]
    df = df.reset_index(drop=True)
    df.to_json('defensive' + link + '.json', orient='records', force_ascii=False)

driver.quit()

IndexError: ignored

In [ ]:
#passing statistics
driver = webdriver.Chrome('chromedriver', options=chrome_options)
passing_links = ['https://fbref.com/en/comps/9/1631/passing/2017-2018-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/1640/passing/2017-2018-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/1652/passing/2017-2018-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/1634/passing/2017-2018-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/1632/passing/2017-2018-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/1889/passing/2018-2019-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/1896/passing/2018-2019-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/1886/passing/2018-2019-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/2109/passing/2018-2019-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/2104/passing/2018-2019-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/3232/passing/2019-2020-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/3260/passing/2019-2020-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/3239/passing/2019-2020-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/3248/passing/2019-2020-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/3243/passing/2019-2020-Ligue-1-Stats',
                  'https://fbref.com/en/comps/9/10728/passing/2020-2021-Premier-League-Stats',
                  'https://fbref.com/en/comps/11/10730/passing/2020-2021-Serie-A-Stats',
                  'https://fbref.com/en/comps/12/10731/passing/2020-2021-La-Liga-Stats',
                  'https://fbref.com/en/comps/20/10737/passing/2020-2021-Bundesliga-Stats',
                  'https://fbref.com/en/comps/13/10732/passing/2020-2021-Ligue-1-Stats']

iteration = 0
for link in passing_links:
    page = driver.get(link)
    time.sleep(2)

    #we find the table
    table = driver.find_element(By.XPATH , "//*[@id='stats_passing']")
    table_html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(table_html, "html.parser")

    #here are the headers columns of the dataframe where we will copy the table
    headers = []
    for i in soup.find_all('th'):
        title = i.text
        headers.append(title)
    headers = headers[11:41]
    df = pd.DataFrame(columns=headers)

    #we build the dataframe
    for j in soup.find_all('tr')[2:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        if row:
            length = len(df)
            df.loc[length] = row

    #we assaign the return value of checkname to the name field, we keep only one role
    for i in range(len(df)-1):
        name = df.loc[i, 'Player']
        team = df.loc[i, 'Squad']
        df.loc[i, 'Player'] = checkname(football_players[iteration], name, players_teams_2[iteration], team)
        pos = df.loc[i, 'Pos']
        if (len(pos.split(',')) > 1):
            df.loc[i, 'Pos'] = pos.split(',')[0]

    df = df[['Player', 'Pos', 'Cmp', 'Cmp%', '1/3', 'PPA', 'CrsPA', 'Prog']]
    df.columns = ['Player', 'Pos', 'Cmp', 'Cmp1', 'Cmp2', 'Cmp3', 'Cmp%', 'Cmp%1', 'Cmp%2', 'Cmp%3', '1/3', 'PPA',
                  'CrsPA', 'Prog']

    for i in range(len(df) - 1):
        if df.loc[i, 'Cmp']:
            df.loc[i, 'Cmp'] = float(df.loc[i, 'Cmp'])
        if df.loc[i, 'Cmp%']:
            df.loc[i, 'Cmp%'] = float(df.loc[i, 'Cmp%'])
        if df.loc[i, '1/3']:
            df.loc[i, '1/3'] = float(df.loc[i, '1/3'])
        if df.loc[i, 'PPA']:
            df.loc[i, 'PPA'] = float(df.loc[i, 'PPA'])
        if df.loc[i, 'CrsPA'] :
            df.loc[i, 'CrsPA'] = float(df.loc[i, 'CrsPA'])
        if df.loc[i, 'Prog']:
            df.loc[i, 'Prog'] = float(df.loc[i, 'Prog'])

    iteration = iteration + 1
    df.drop(df[df.Player == ''].index, inplace=True)
    #we keep only the statistics that we are interested in
    df = df[['Player', 'Pos', 'Cmp', 'Cmp%', '1/3', 'PPA', 'CrsPA', 'Prog']]

    df.drop(df[df.Player == ''].index, inplace=True)
    print(len(df))
    link = link.split('/')[-1]
    df = df.reset_index(drop=True)
    df.to_json('passing' + link + '.json', orient = 'records', force_ascii=False)

driver.quit()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69:

310
289
303


# Data modelling
<p>Once the data has been collected, we can determine the most suitable structure for our database. Given the structure of the downloaded data, the most suitable NoSQL model seems to be the document-based one. The logical structure is the following. We define a collection for each league of a certain season. The seasons range from 2017/2018 to 2020/2021, the leagues considered are the Premier League, Serie A, La Liga, Bundesliga and Ligue 1. Within each collection we have one document per player. Within each document there are the following fields: player name, age, nationality, height, weight, position, team. In addition, each document contains four other documents: general_stats containing the number of games played, total minutes played, yellow cards and red cards, offensive_stats containing statistics downloaded from Understat, passing_stats and defensive_stats containing statistics downloaded from Fbref. </p>



#Data aggregation
<p>We use the name of the players as a reference field for the aggregation of the different datasets; with the procedure explained above we have ensured that this field is suitable for the purpose. Analysing the number of players in the files obtained, we observe that the files downloaded from api-football generally have fewer players. This is due to the greater difference in the name format (highlighted earlier) between Understat and api-football than the one between Understat and Fbref. We decide to start with the data downloaded from api-football and aggregate it with that obtained from Understat and Fbref.</p>

# Pymongo
<p>We choose MongoDB as our document-based DBMS. To load and aggregate data we use pymongo, a python package for interacting with MongoDB databases.
To aggregate the data we have more options. We can load all the json files obtained in a database creating a collection for each file and then use the option of aggregation between collections i.e. look up, as reference field the name of the players. The problem with this option is that the results of the aggregation are provided in the form of arrays, whereas we would rather to have the various game statistics as an Object type because it facilitates the navigation within the objects and because it is more logical, since all the  generated arrays would have only one element. Therefore we decide to load only the files obtained from api-football.com and then add the various statistics using the update_one function, which allows us to obtain Object types instead of Arrays. Using the input parameters of the function and the name of the players, we will carry out the aggregation.</p>.
<p>We load the files with data from api-football.com so that we have a collection for each season of a certain league and an object for each player. For each season and league we load the contents of the corresponding file obtained from Understat into a list of python dictionaries. We load the statistics of the i-th dictionary of the list into the MongoDB object, whose value of the field indicating the name of the player is equal to that of the dictionary. We proceed in the same way with the files downloaded from Fbref. Here is the code. </p>

In [ ]:
!pip install pymongo

In [ ]:
import json
import os
from pymongo import MongoClient

# we add in the db the data downloaded from api-football.com
myclient = MongoClient()
db = myclient["Football_Players"]

seasons = ['2017','2018','2019','2020']
leagues = ['epl','Serie A','La Liga','Bundesliga','Ligue 1']

for season in seasons:
    for league in leagues:
        Collection = db[season+league]

        with open(season + league + '_players.json', encoding='utf-8') as file:
            file_data = json.load(file)

        if isinstance(file_data, list):
            Collection.insert_many(file_data)
        else:
            Collection.insert_one(file_data)

for season in seasons:
    for league in leagues:
        with open('stats_'+ league + season + '_players.json', encoding='utf-8') as file:
            file_data = json.load(file)

        Collection = db[season + league]
        for i in range(len(file_data)):
            dizionario = file_data[i]
            Collection.update_one({'name': dizionario['player_name']}, {'$set': {'team': dizionario['team_title']}})

#we add the position (from Fbref)
seasons_fbref = ['2017-2018','2018-2019','2019-2020','2020-2021']
leagues_fbref = ['Premier-League','Serie-A','La-Liga','Bundesliga','Ligue-1']

for season, season_fbref in zip(seasons, seasons_fbref):
    for league, league_fbref in zip(leagues, leagues_fbref):
        with open('defensive' + season_fbref + '-' + league_fbref + '-Stats.json', encoding='utf-8') as file:
            file_data = json.load(file)

        Collection = db[season+league]
        for i in range(len(file_data)):
            dizionario = file_data[i]
            Collection.update_one({'name': dizionario['Player']}, {'$set': {'position': dizionario['Pos']}})

#we add the general and offensive statistics
for season in seasons:
    for league in leagues:
        with open('stats_'+ league + season + '_players.json', encoding='utf-8') as file:
            file_data = json.load(file)

        Collection = db[season + league]
        for i in range(len(file_data)):
            dizionario = file_data[i]
            keys_to_extract = ["games", "time", "red_cards", "yellow_cards"]
            dizionario_2 = {key: dizionario[key] for key in keys_to_extract}
            Collection.update_one({'name': dizionario['player_name']}, {'$set': {'general_stats': dizionario_2}})
            Collection.update_one({'name': dizionario['player_name']}, {'$set': {'offensive_stats': dizionario}})


#we add the defensive statistics
for season, season_fbref in zip(seasons, seasons_fbref):
    for league, league_fbref in zip(leagues, leagues_fbref):
        with open('defensive' + season_fbref + '-' + league_fbref + '-Stats.json', encoding='utf-8') as file:
            file_data = json.load(file)

        Collection = db[season+league]
        for i in range(len(file_data)):
            dizionario = file_data[i]
            Collection.update_one({'name': dizionario['Player']}, {'$set': {'defensive_stats': dizionario}})

#we add the passing statistics
for season, season_fbref in zip(seasons, seasons_fbref):
    for league, league_fbref in zip(leagues, leagues_fbref):
        with open('passing' + season_fbref + '-' + league_fbref + '-Stats.json', encoding='utf-8') as file:
            file_data = json.load(file)

        Collection = db[season+league]
        for i in range(len(file_data)):
            dizionario = file_data[i]
            Collection.update_one({'name': dizionario['Player']}, {'$set': {'passing_stats': dizionario}})

ServerSelectionTimeoutError: ignored

We must now delete some data in order to arrive at the desired structure. We delete the id, injured and photo fields because we are not interested in. We delete the fields id, player_name, games, time, yellow_cards, red_cards, position and team_title from offensive_stats, because we have placed them in general_stats. We delete the fields Player and Pos from passing_stats and defensive_stats, because we have inserted them as general fields. Now we have the desired database.

In [ ]:
for season in seasons:
    for league in leagues:
        Collection = db[season + league]
        Collection.update_many({}, {'$unset': {'id': '', 'injured': '', 'photo': '', 'offensive_stats.id': '',
                                               'offensive_stats.player_name': '', 'offensive_stats.games': '',
                                               'offensive_stas.time': '', 'offensive_stats.yellow_cards': '',
                                               'offensive_stats.red_cards': '', 'offensive_stats.position': '',
                                               'offensive_stas.team_title': '', 'passing_stats.Player': '',
                                               'passing_stats.Pos': '', 'defensive_stats.Player': '',
                                               'defensive_stats.Pos': ''}})

In [ ]:
seasons = ['2017', '2018','2019','2020']
leagues = ['epl','Serie A','La Liga', 'Ligue 1']
leagues_names=['PremierLeague', 'SerieA', 'LaLiga', 'Ligue1']

for season in seasons:
    for league, name in zip(leagues, leagues_names):
        Collection = db[season+league]
        Collection.rename(season+name)